In [1]:
import numpy as np
# importing decimal module to control significant figures
import decimal

In [2]:
def toDecimal(array) :
    # flatten the array convert floats to decimal and reshape to original shape
    originalShape = array.shape
    array = array.flatten().astype(object) 
    for i in range(array.size) :
        array[i] = decimal.Decimal(str(array[i]))
    array = array.reshape(originalShape)   
    return array


def intializeContext(significantFigs,rounding) :
    #Intialize the context with specified signFigs and rounding/chopping
    decimal.getcontext().prec = significantFigs 
    if( rounding ) :
        decimal.getcontext().rounding = decimal.ROUND_HALF_UP
    else :
        decimal.getcontext().rounding = decimal.ROUND_DOWN 
        
def toFloats (array) :
    #Convert the decimals back to float
    originalShape = array.shape
    array = array.flatten().astype(object)
    for i in range(array.size) :
        array[i] = float(array[i]) 
    array = array.reshape(originalShape)    
    return array            
    

In [3]:
def isDiagonalyDominant(A) :
    #Check if Matrix A is diagonaly dominant 
    dim = A.shape
    # Flags to check system
    at_least_one_strictly_greater = None 
    if(dim[0] != dim[1]) : raise Exception("Matrix must be square")
    for i in range(dim[0]) :
        nonDiagonalSum = 0 
        for j in range(dim[0]) :
            if(j==i) : continue 
            nonDiagonalSum += abs(A[i][j]) 
        if(A[i][i] > nonDiagonalSum) :
            at_least_one_strictly_greater = True
        elif(A[i][i] == nonDiagonalSum) :
            if(at_least_one_strictly_greater == None) : at_least_one_strictly_greater = False
        else :
            return False    
    return at_least_one_strictly_greater           

In [4]:
# GaussSeidal with relaxation for improved convergence
# the parameter n can be removed as we can get the no of vars from the npArray
def GaussSeidal(A,b,n,x,maxIterations,ErrorTolerance,relax , significantFigs = 7 , rounding = True) :
    # Setting up signifcant figs and rounding/chopping
    intializeContext(significantFigs,rounding)
    
    #converting floats to decimals 
    A = toDecimal(A)
    b = toDecimal(b)
    x = toDecimal(x)
    relax = decimal.Decimal(str(relax))
    # Normalizaing the equations such that a(i,i) = 1
    for i in range(n) :
        diag = A[i][i]
        for j in range(n) :
            A[i][j] = A[i][j] / diag
        b[i] = b[i] / diag
    # Calculating first iteration before applying relaxation    
    for i in range(n) :
        sum = b[i]
        for j in range(n) :
            if(i==j) :
                continue
            sum -= A[i][j] * x[j]
        x[i] = sum 
    iteration = 2
    # Loop until convergence or max iterations reached 
    while (True) :
        belowTolerance = True
        for i in range(n) :
            oldX = x[i]
            sum = b[i]
            for j in range(n) :
                if(i==j) :
                    continue
                sum -= A[i][j] * x[j]
            x[i] = relax*sum + (1-relax)*oldX
            if (belowTolerance and x[i] != 0) :
                estimatedError = abs(float((x[i]-oldX)/x[i]))
                estimatedError = float(estimatedError)
                if(estimatedError > ErrorTolerance):
                    belowTolerance = False
        iteration+=1
        if(belowTolerance or iteration >= maxIterations) :
            break
    return toFloats(x)

In [5]:
# GaussSeidel with relaxation for improved convergence
# the parameter n can be removed as we can get the no of vars from the npArray
def GaussSeidel_noNorm(A,b,n,x,maxIterations,ErrorTolerance,relax , significantFigs = 7 , rounding = True) :
    # Setting up signifcant figs and rounding/chopping
    intializeContext(significantFigs,rounding)
    
    #converting floats to decimals 
    A = toDecimal(A)
    b = toDecimal(b)
    x = toDecimal(x)
    relax = decimal.Decimal(str(relax))
    # Calculating first iteration before applying relaxation    
    for i in range(n) :
        sum = b[i]
        for j in range(n) :
            if(i==j) :
                continue
            sum -= A[i][j] * x[j]
        x[i] = sum/A[i][i]
    iteration = 2
    # Loop until convergence or max iterations reached 
    while (True) :
        belowTolerance = True
        for i in range(n) :
            oldX = x[i]
            sum = b[i]
            for j in range(n) :
                if(i==j) :
                    continue
                sum -= A[i][j] * x[j]
            x[i] = relax*sum/A[i][i] + (1-relax)*oldX
            if (belowTolerance and x[i] != 0) :
                estimatedError = abs(float((x[i]-oldX)/x[i]))
                estimatedError = float(estimatedError)
                if(estimatedError > ErrorTolerance):
                    belowTolerance = False
        iteration+=1
        if(belowTolerance or iteration >= maxIterations) :
            break
    return toFloats(x)

In [6]:
def test_GaussSeidel():
    tests = []

    # Example 1: 3x3 system
    A1 = np.array([[4, -1, 0],
                   [-1, 4, -1],
                   [0, -1, 3]], dtype=float)
    b1 = np.array([15, 10, 10], dtype=float)
    x0_1 = np.zeros(3)
    tests.append((A1, b1, x0_1))

    # Example 2: 4x4 diagonally dominant system
    A2 = np.array([[10, -1, 2, 0],
                   [-1, 11, -1, 3],
                   [2, -1, 10, -1],
                   [0, 3, -1, 8]], dtype=float)
    b2 = np.array([6, 25, -11, 15], dtype=float)
    x0_2 = np.zeros(4)
    tests.append((A2, b2, x0_2))

    # Parameters
    maxIter = 100
    tol = 1e-6
    relax = 1.0
    sigFigs = 4
    rounding = decimal.ROUND_DOWN

    for idx, (A, b, x0) in enumerate(tests):
        n = A.shape[0]
        x = GaussSeidal(A, b, n, x0, maxIter, tol, relax, sigFigs, rounding)
        x_np = np.linalg.solve(A, b)  # reference solution
        error = np.linalg.norm(x - x_np)
        print(f"Test {idx+1} | Solution: {x} | NumPy: {x_np} | Error Norm: {error:.2e}")

In [7]:
test_GaussSeidel()


Test 1 | Solution: [5.0 5.0 5.0] | NumPy: [5. 5. 5.] | Error Norm: 0.00e+00
Test 2 | Solution: [1.0 2.0 -1.0 1.0] | NumPy: [ 1.  2. -1.  1.] | Error Norm: 0.00e+00


In [8]:
def spicy_test():
    # 5x5 system, nearly diagonally dominant, tricky for convergence
    A = np.array([[1.01, 0.99, 0.0, 0.0, 0.0],
                  [0.5, 1.0, 0.5, 0.0, 0.0],
                  [0.0, 0.5, 1.0, 0.5, 0.0],
                  [0.0, 0.0, 0.5, 1.0, 0.5],
                  [0.0, 0.0, 0.0, 0.99, 1.01]], dtype=float)

    b = np.array([2.0, 1.5, 1.5, 1.5, 2.0], dtype=float)
    x0 = np.zeros(5)

    maxIter = 500
    tol = 1e-6

    # Try under-relaxation
    relax = 0.8
    sigFigs = 10
    rounding = False  # chopping

    x = GaussSeidal(A, b, 5, x0, maxIter, tol, relax, sigFigs, rounding)
    
    # Reference solution using NumPy
    x_np = np.linalg.solve(A, b)

    print("Spicy Test - Nearly Diagonally Dominant System")
    print("Gauss-Seidel solution:", x)
    print("NumPy solution       :", x_np)
    print("Error norm           :", np.linalg.norm(x - x_np))

In [9]:
spicy_test()

Spicy Test - Nearly Diagonally Dominant System
Gauss-Seidel solution: [24.84163867 -23.33093619 24.82940653 -23.33700427 24.85359391]
NumPy solution       : [ 25.75 -24.25  25.75 -24.25  25.75]
Error norm           : 2.038233551604123


In [10]:
def spicy_test_2():
    # Ill-conditioned 4x4 system
    # Some elements very small, others large → testing numerical stability
    A = np.array([[1e-5, 2, 3, 4],
                  [2, 1e-5, 3, 4],
                  [3, 2, 1e-5, 4],
                  [4, 2, 3, 1e-5]], dtype=float)

    b = np.array([10, 20, 30, 40], dtype=float)
    x0 = np.zeros(4)

    maxIter = 100
    tol = 1e-8

    # Relaxation factor
    relax = 0.5
    sigFigs = 10
    rounding = False  # chopping mode

    x = GaussSeidal(A, b, 4, x0, maxIter, tol, relax, sigFigs, rounding)
    x_np = np.linalg.solve(A, b)

    print("Spicier Test - Ill-Conditioned System")
    print("Gauss-Seidel solution:", x)
    print("NumPy solution       :", x_np)
    print("Error norm           :", np.linalg.norm(x - x_np))

In [11]:
spicy_test_2()

Spicier Test - Ill-Conditioned System
Gauss-Seidel solution: [inf -inf inf -inf]
NumPy solution       : [7.77779043 2.77776543 1.11110154 0.27777168]
Error norm           : inf


In [12]:
def spicy_test_2_noNorm():
    # Ill-conditioned 4x4 system
    # Some elements very small, others large → testing numerical stability
    A = np.array([[1e-5, 2, 3, 4],
                  [2, 1e-5, 3, 4],
                  [3, 2, 1e-5, 4],
                  [4, 2, 3, 1e-5]], dtype=float)

    b = np.array([10, 20, 30, 40], dtype=float)
    x0 = np.zeros(4)

    maxIter = 100
    tol = 1e-8

    # Relaxation factor
    relax = 0.5
    sigFigs = 10
    rounding = False  # chopping mode

    x = GaussSeidel_noNorm(A, b, 4, x0, maxIter, tol, relax, sigFigs, rounding)
    x_np = np.linalg.solve(A, b)

    print("Spicier Test - Ill-Conditioned System")
    print("Gauss-Seidel solution:", x)
    print("NumPy solution       :", x_np)
    print("Error norm           :", np.linalg.norm(x - x_np))

In [13]:
spicy_test_2_noNorm()

Spicier Test - Ill-Conditioned System
Gauss-Seidel solution: [inf -inf inf -inf]
NumPy solution       : [7.77779043 2.77776543 1.11110154 0.27777168]
Error norm           : inf


In [14]:

def safe_spicy_test():
    # 5x5 diagonally dominant system (convergent)
    A = np.array([[10, 2, 1, 1, 0],
                  [1, 15, 2, 0, 1],
                  [2, 1, 20, 2, 2],
                  [0, 1, 2, 25, 3],
                  [1, 0, 1, 2, 30]], dtype=float)

    b = np.array([14, 18, 24, 30, 36], dtype=float)
    x0 = np.zeros(5)

    maxIter = 500
    tol = 1e-6
    relax = 0.9           # under-relaxation
    sigFigs = 6
    rounding = False      # chopping

    x = GaussSeidel_noNorm(A, b, 5, x0, maxIter, tol, relax, sigFigs, rounding)
    x_np = np.linalg.solve(A, b)

    print("Safe Spicy Test - Diagonally Dominant 5x5 System")
    print("Gauss-Seidel solution:", x)
    print("NumPy solution       :", x_np)
    print("Error norm           :", np.linalg.norm(x - x_np))

safe_spicy_test()

Safe Spicy Test - Diagonally Dominant 5x5 System
Gauss-Seidel solution: [1.02943 0.947044 0.845816 0.965675 1.0731]
NumPy solution       : [1.02943907 0.94705331 0.84582432 0.9656784  1.07311266]
Error norm           : 2.024725919258251e-05
